# Preprocessing, Step 0 bis Step 7 + Regression


In [14]:
import os
from pathlib import Path
import seaborn as sns  
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import missingno as msno
from sklearn import linear_model
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

In [15]:
df = pd.read_csv('../../data/raw/triathlon_watch_test_data_final.csv')
# Rename columns for better handling
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df.info

<bound method DataFrame.info of         id  age_of_customer     sex       ctry         town  \
0    C0001             60.0   Other    Germany      Hamburg   
1    C0002             53.0    Male    Germayn       Berlin   
2    C0003             30.0   Other         UK       London   
3    C0004             24.0   Other    Germany      Hamburg   
4    C0005             53.0    Male        USA  Los Angeles   
..     ...              ...     ...        ...          ...   
495  C0495             43.0  Female         UK   Birmingham   
496  C0496             60.0  Female        USA      Chicago   
497  C0498             44.0    Male      India        Delhi   
498  C0499             46.0  Female         UK       London   
499  C0500             64.0   Other  Australia     Brisbane   

     swimming_hours_per_week  biking_hours_per_week  running_hours_per_week  \
0                       1.80                   4.44                    0.34   
1                       1.77                   5.42  

In [16]:
# Spalte ID wird das führende "C" entfernt und die führenden Nullen
df["id"] = df["id"].dropna().str.replace("C", "", regex=True).astype(int)

In [17]:
# Sicherstellen, dass die Spalte als Datumsformat erkannt wird
df["most_current_software_update"] = pd.to_datetime(df["most_current_software_update"], format="%Y-%m-%d")
# ✅ Differenz zum jüngsten Datum berechnen (in Tagen)
latest_date = df["most_current_software_update"].max()
df["days_since_latest_update"] = (latest_date - df["most_current_software_update"]).dt.days
# Ergebnis ausgeben
print(df[["most_current_software_update", "days_since_latest_update"]].head())
df = df.drop(columns=["most_current_software_update"])

  most_current_software_update  days_since_latest_update
0                   2023-12-05                       391
1                   2024-04-23                       251
2                   2024-12-30                         0
3                   2023-10-30                       427
4                   2024-12-23                         7


In [18]:
numeric=df.select_dtypes(include=['float64']).columns.tolist() #list of numeric columns
categorical = df.select_dtypes(include=['object']).columns.to_list() #append all categorical columns to a list

In [19]:
print(df.dtypes)

id                               float64
age_of_customer                  float64
sex                               object
ctry                              object
town                              object
swimming_hours_per_week          float64
biking_hours_per_week            float64
running_hours_per_week           float64
total_training_hours_per_week    float64
vo2_max                          float64
10k_running_time_prediction      float64
calories_burned_per_week         float64
support_cases_of_customer          int64
customer_years                     int64
goal_of_training                  object
preferred_training_daytime        object
subscription_type                 object
color_of_watch                    object
synchronisation                   object
user_of_latest_model               int64
days_since_latest_update           int64
dtype: object


**bis hierin wurde nur ganz einfache Veränderungen vollzogen. df ist nun die Grundlage für die weietren Steps**

# Step 0 Simple Preprocessing

In [20]:
df_step0 =df.copy()

In [21]:

target_column = "user_of_latest_model" if "user_of_latest_model" in df_step0.columns else df.columns[-1]
# Zeilen mit fehlenden Werten in der Zielvariable entfernen
df_step0 = df_step0.dropna(subset=[target_column])
# Zeilen mit mehr als zwei fehlenden Werten entfernen
df_step0 = df_step0[df_step0.isnull().sum(axis=1) <= 2]

In [22]:
# Fehlende Werte analysieren
missing_values = df_step0.isnull().sum()
missing_values = missing_values[missing_values > 0]

# Fehlende Werte durch Durchschnitt (numerisch) oder Modus (kategorisch) ersetzen
#df_step0 = df_step0.copy()

numeric_cols = df_step0.select_dtypes(include=[np.number]).columns
categorical_cols = df_step0.select_dtypes(exclude=[np.number]).columns

for col in numeric_cols:
    df_step0[col] = df_step0[col].fillna(df_step0[col].mean())

for col in categorical_cols:
    df_step0[col] = df_step0[col].fillna(df_step0[col].mode()[0])

# Speichern der bereinigten Daten
output_path = "triathlon_watch_training_data_step0.csv"
df_step0.to_csv(output_path, index=False)

# Ergebnis anzeigen
print("Step 0 abgeschlossen. Bereinigte Daten gespeichert unter:", output_path)
print(df_step0.head())



Step 0 abgeschlossen. Bereinigte Daten gespeichert unter: triathlon_watch_training_data_step0.csv
    id  age_of_customer    sex     ctry         town  swimming_hours_per_week  \
0  1.0             60.0  Other  Germany      Hamburg                     1.80   
1  2.0             53.0   Male  Germayn       Berlin                     1.77   
2  3.0             30.0  Other       UK       London                     4.05   
3  4.0             24.0  Other  Germany      Hamburg                     3.22   
4  5.0             53.0   Male      USA  Los Angeles                     3.15   

   biking_hours_per_week  running_hours_per_week  \
0                   4.44                    0.34   
1                   5.42                    5.60   
2                   6.98                    4.03   
3                  10.54                    4.23   
4                   3.03                    5.40   

   total_training_hours_per_week  vo2_max  ...  calories_burned_per_week  \
0                         

In [23]:
numerical_columns = df_step0.select_dtypes(include=['number']).columns.tolist()
categorical_columns = df_step0.select_dtypes(exclude=['number']).columns.tolist()

print("📊 Numerische Spalten:", numerical_columns)
print("🔠 Kategoriale Spalten:", categorical_columns)


📊 Numerische Spalten: ['id', 'age_of_customer', 'swimming_hours_per_week', 'biking_hours_per_week', 'running_hours_per_week', 'total_training_hours_per_week', 'vo2_max', '10k_running_time_prediction', 'calories_burned_per_week', 'support_cases_of_customer', 'customer_years', 'user_of_latest_model', 'days_since_latest_update']
🔠 Kategoriale Spalten: ['sex', 'ctry', 'town', 'goal_of_training', 'preferred_training_daytime', 'subscription_type', 'color_of_watch', 'synchronisation']


## Regression als Funktion

In [24]:
df_score_test = pd.DataFrame(columns=['arbeitsschritt', 'Accuracy', 'F1-Score'])

In [ ]:
import joblib
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score

def run_logistic_regression(df, arbeitsschritt):
    global df_score_test

    # **Extrahiere die erste Zahl aus arbeitsschritt**
    step_number = str(arbeitsschritt).split()[0]  # Ersten Wert extrahieren
    print(f"Arbeitsschritt extrahierte Nummer: {step_number}")

    # Trennen von Features und Zielvariable
    target_column = 'user_of_latest_model'
    X_test = df.drop(columns=[target_column])
    y_test = df[target_column]

    # Identifikation der kategorischen Variablen
    categorical_cols = X_test.select_dtypes(include=['object', 'category']).columns

   # One-Hot-Encoding für kategoriale Variablen
    encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    X_encoded = pd.DataFrame(
        encoder.fit_transform(X[categorical_cols]),
        columns=encoder.get_feature_names_out(categorical_cols),
        index=X.index
    )

        # Numerische Spalten beibehalten
        X_numeric = X_test.drop(columns=categorical_cols)

        # Zusammenfügen der numerischen und encodierten Daten
        X_final = pd.concat([X_numeric, X_encoded], axis=1)
    except FileNotFoundError:
        print(f"FEHLER: Encoder-Datei {encoder_filename} nicht gefunden! Überspringe OneHotEncoding.")
        X_final = X_test  # Falls kein Encoding nötig

    # **Laden des trainierten Modells**
    model_filename = f"model_parameters_Andreas_{step_number}.pkl"
    try:
        model = joblib.load(model_filename)
        print(f"Geladenes Modell: {model_filename}")
    except FileNotFoundError:
        print(f"FEHLER: Modell-Datei {model_filename} nicht gefunden!")
        return None

    # **Vorhersagen auf dem Testdatensatz**
    y_pred = model.predict(X_final)

    # **Metriken berechnen**
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"Testset - Accuracy: {accuracy:.2f}, F1-Score: {f1:.2f}")

    # Ergebnisse in DataFrame speichern
    new_row = pd.DataFrame({
        "arbeitsschritt": [arbeitsschritt],
        "Test_Accuracy": [accuracy],
        "Test_F1-Score": [f1]
    })
    df_score_test = pd.concat([df_score_test, new_row], ignore_index=True)

    return df_score_test




In [ ]:
run_logistic_regression(df_step0, '0 Simple Preprocessing')
df_score_test

# Step 1 Data Quality Correction

In [ ]:
df_step1 = df_step0.copy()

In [ ]:
# Überblick über das Feature "ctry"
ctry_overview = df_step1['ctry'].value_counts()
# Ergebnis anzeigen
print(ctry_overview)

# Korrigiere den Ländernamen "Germayn" zu "Germany"
df_step1['ctry'] = df_step1['ctry'].replace('Germayn', 'Germany')

# Überprüfen, ob die Änderung erfolgreich war
ctry_overview = df_step1['ctry'].value_counts()
print(ctry_overview)

In [ ]:
# Doppelte Datensätze anzeigen
duplicate_rows = df_step1[df_step1.duplicated()]
print("Doppelte Datensätze:")
print(duplicate_rows)

# Doppelte Datensätze entfernen
df_step1 = df_step1.drop_duplicates()
print("Doppelte Datensätze wurden entfernt.")

In [ ]:
run_logistic_regression(df_step1, '1 Data Quality Correction')
df_score_test

# Step 2 Missing Value Handling

In [ ]:
df_step2 = df.copy()

In [ ]:
#Listwise deletion all rows with missing values in the column 'user_of_latest_model'
df_step2=df_step2.drop(df_step2[df_step2['user_of_latest_model'].isnull()].index)
msno.matrix(df_step2)

In [ ]:
df_step2[numeric].isnull().sum()

In [ ]:
correlation_with_A = df_step2[numeric].corr()['biking_hours_per_week'].drop('biking_hours_per_week')  # Entferne die Korrelation mit sich selbst

# Ausgabe der Korrelation
print(correlation_with_A)

In [ ]:
#  Iterative Imputation for numeric columns
imputer = IterativeImputer(max_iter=30, tol=1e-2, random_state=42, initial_strategy='median') #initial all missing values were replaced by median
df_step2_numeric_imputed = pd.DataFrame(imputer.fit_transform(df_step2[numeric])) # creating a new dataframe with imputed values

df_step2_numeric_imputed.index = df_step2.index # adapt index of new dataframe to index of original dataframe
df_step2[numeric] = df_step2_numeric_imputed # replace numeric columns in original dataframe with imputed values

print(df_step2[numeric].isnull().sum()) # check if all missing values in numeric columns were imputed

In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

# 1. Kategorische Spalten in numerische Werte umwandeln
label_encoders = {}  # Dictionary, um die LabelEncoder zu speichern

for col in categorical:
    if df_step2[col].dtype == 'object':  # Überprüfen, ob es sich um eine kategorische Spalte handelt
        encoder = LabelEncoder()
        df_step2[col] = encoder.fit_transform(df_step2[col].fillna('Missing'))  # Umwandlung und fehlende Werte ersetzen
        label_encoders[col] = encoder  # Speichern des Encoders

# 2. KNN-Imputation auf den numerischen Werten anwenden
imputer = KNNImputer(n_neighbors=2)  # Anzahl der Nachbarn (k) einstellen
df_imputed_categorical = pd.DataFrame(imputer.fit_transform(df_step2[categorical]), columns=categorical)

# 3. Imputierte Werte zurück in kategorische Werte umwandeln
for col, encoder in label_encoders.items():
    df_imputed_categorical[col] = encoder.inverse_transform(df_imputed_categorical[col].round().astype(int))  # Rückumwandlung

# Ausgabe des imputierten DataFrames
df_imputed_categorical

df_imputed_categorical.index = df_step2.index # adapt index of new dataframe to index of original dataframe
df_step2[categorical] = df_imputed_categorical # replace numeric columns in original dataframe with imputed values

print(df_step2.isnull().sum()) # check if all missing values in numeric columns were imputed

In [ ]:
run_logistic_regression(df_step2, '2 Missing Value Handling')
df_score_test

# Step 3 Outliers

In [ ]:
df_step3 = df_step0.copy()

#  Detection of outliers with IQR-method
def detect_outliers_iqr(df):
    df_outliers = df.copy()
    for col in df.select_dtypes(include=np.number):  # Nur numerische Spalten
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_outliers[col] = df[col].apply(lambda x: np.nan if x < lower_bound or x > upper_bound else x)
    return df_outliers


#  Replacement of outliers mit NaN
data_numeric_no_outliers = detect_outliers_iqr(df_step3[numeric])

data_numeric_no_outliers.isnull().sum()

In [ ]:
#  Iterative Imputation for numeric columns
imputer = IterativeImputer(max_iter=30, tol=1e-2, random_state=42, initial_strategy='median') #initial all missing values were replaced by median
data_numeric_imputed = pd.DataFrame(imputer.fit_transform(data_numeric_no_outliers)) # creating a new dataframe with imputed values

data_numeric_imputed.index = df_step3.index # adapt index of new dataframe to index of original dataframe
#df_imputed_numeric = df_step3.copy() # create a copy of the original dataframe
df_step3[numeric] = data_numeric_imputed # replace numeric columns in original dataframe with imputed values

print(df_step3[numeric].isnull().sum()) # check if all missing values in numeric columns were imputed

In [ ]:
run_logistic_regression(df_step3, '3 Outliers')
df_score_test

# Step 4 Transformation Normalverteilung

In [ ]:
df_step4 = df_step0.copy()

In [ ]:
# Showing the distribution of the numerical data with histograms
%matplotlib inline
hist = df_step4[numeric].hist(bins=30,figsize=(10, 10))

In [ ]:
numeric_ohne_target = numeric.copy()
numeric_ohne_target.remove('user_of_latest_model')

Der Hauptzweck dieser Transformation ist:
Umwandlung nicht-normalverteilter Daten in normalverteilte Daten
Reduzierung des Einflusses von Ausreißern
Verbesserung der Performance von Machine Learning Modellen, die von normalverteilten Daten profitieren
Standardisierung der Datenverteilung über alle numerischen Features

In [ ]:
from sklearn.preprocessing import QuantileTransformer

#Erstellt einen Transformer, der die Daten in eine Normalverteilung überführt
qt = QuantileTransformer(n_quantiles=25, output_distribution='normal', random_state=0)

#Lernt die Transformation aus den Daten und wendet sie direkt an
#Transformiert alle numerischen Spalten außer 'user_of_latest_model'
#Das Ergebnis ist eine NumPy-Array mit den transformierten Werten
trans_x = qt.fit_transform(df_step4[numeric_ohne_target])  

#Wandelt die transformierte Array wieder in einen DataFrame um
#Ersetzt die ursprünglichen Werte im DataFrame mit den transformierten Werten
#Behält die Index-Struktur bei
df_step4[numeric_ohne_target] = pd.DataFrame(trans_x, columns=numeric_ohne_target, index=df_step4.index)

# Plot histograms for each numerical column to visualize the distribution
df_step4[numeric].hist(bins=20, figsize=(10, 10))
plt.show()


In [ ]:
run_logistic_regression(df_step4, '4 Transformation Normalverteilung')
df_score_test

# Step 5 Power Transformation

In [ ]:
df_step5 = df_step0.copy()

In [ ]:
from sklearn.preprocessing import PowerTransformer
import matplotlib.pyplot as plt

# Initialize PowerTransformer (Yeo-Johnson is default, Box-Cox requires only positive data)
pt = PowerTransformer(method='yeo-johnson', standardize=True)  

# Apply the transformation to all numerical columns
trans_x = pt.fit_transform(df_step5[numeric_ohne_target])  

# Convert the transformed array back to a DataFrame and replace original numerical columns
df_step5[numeric_ohne_target] = pd.DataFrame(trans_x, columns=numeric_ohne_target, index=df_step5.index)

# Plot histograms for each numerical column to visualize the distribution
df_step5[numeric].hist(bins=20, figsize=(10, 10))
plt.show()

In [ ]:
run_logistic_regression(df_step5, '5 Power Transformation')
df_score_test

# Step 6 Min-Max-Scaler

In [ ]:
df_step6 = df_step0.copy()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialisiere den MinMaxScaler
scaler = MinMaxScaler()

# Transformiere die numerischen Spalten (ohne das Label)
scaled_features = scaler.fit_transform(df_step6[numeric_ohne_target])

# Konvertiere die transformierten Daten zurück in einen DataFrame mit den ursprünglichen Spaltennamen
df_step6[numeric_ohne_target] = pd.DataFrame(scaled_features, 
                                            columns=numeric_ohne_target, 
                                            index=df_step6.index)

# Visualisiere die Verteilung der skalierten Daten
df_step6[numeric].hist(bins=20, figsize=(10, 10))
plt.show()

# Optional: Überprüfung der Skalierung
print("\nMin-Max Werte nach der Skalierung:")
for column in numeric_ohne_target:
    print(f"{column}:")
    print(f"Min: {df_step6[column].min():.2f}")
    print(f"Max: {df_step6[column].max():.2f}\n")

In [ ]:
run_logistic_regression(df_step6, '6 MIN-MAX-Scaler')
df_score_test

# Step 7 Standard-Scaler

In [ ]:
df_step7 = df_step0.copy()

StandardScaler transformiert die Daten so, dass:
Mittelwert = 0
Standardabweichung = 1
Die Werte sind nicht auf einen bestimmten Bereich beschränkt
Die Transformation erfolgt nach der Formel: z = (x - μ) / σ

Erstellt eine Kopie von df_step4
Führt die Standardisierung der numerischen Features durch
Behält die Zielvariable unverändert
Visualisiert die Verteilungen
Überprüft die erfolgreiche Standardisierung durch Ausgabe von Mittelwert und Standardabweichung

Die standardisierten Werte sollten nun:

Einen Mittelwert nahe 0 haben
Eine Standardabweichung nahe 1 haben
Die ursprüngliche Verteilungsform beibehalten

In [ ]:
# Initialisiere den StandardScaler
scaler = StandardScaler()

# Transformiere die numerischen Spalten (ohne das Label)
scaled_features = scaler.fit_transform(df_step7[numeric_ohne_target])

# Konvertiere die transformierten Daten zurück in einen DataFrame
df_step7[numeric_ohne_target] = pd.DataFrame(scaled_features, 
                                            columns=numeric_ohne_target, 
                                            index=df_step7.index)

# Visualisiere die Verteilung
plt.figure(figsize=(10, 10))
df_step7[numeric].hist(bins=20)
plt.tight_layout()
plt.show()

# Überprüfe die Standardisierung
print("\nÜberprüfung der Standardisierung (Mean ≈ 0, Std ≈ 1):")
for column in numeric_ohne_target:
    print(f"\n{column}:")
    print(f"Mittelwert: {df_step7[column].mean():.4f}")
    print(f"Standardabweichung: {df_step7[column].std():.4f}")

In [ ]:
run_logistic_regression(df_step7, '7 Standard-Scaler')
df_score_test

# Step 8 Ordinal Encoding

In [ ]:
df_step8 = df_step0.copy()

Beim Ordinal Encoding werden kategorische Variablen in numerische Werte umgewandelt, wobei jeder Kategorie eine eindeutige Zahl zugeordnet wird. Dies ist besonders sinnvoll, wenn die Kategorien eine natürliche Ordnung oder Rangfolge haben.

In [ ]:
# Alle kategorischen Spalten im DataFrame anzeigen
categorical_columns = df_step8.select_dtypes(include=['object']).columns
print("Kategorische Spalten im DataFrame:")
print(categorical_columns.tolist())

# Für jede kategorische Spalte die unique Werte anzeigen
for col in categorical_columns:
    print(f"\nUnique Werte in {col}:")
    print(df_step8[col].unique())

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# ✅ Spalten mit natürlicher Ordnung für Ordinal Encoding
ordinal_columns = ['subscription_type', 'goal_of_training', 'preferred_training_daytime']

# Mapping für subscription_type (manuelles Ordinal Encoding)
subscription_mapping = {'Free': 0, 'Basic': 1, 'Premium': 2}
df_step8['subscription_type'] = df_step8['subscription_type'].map(subscription_mapping)

# ✅ Manuelle Reihenfolge für OrdinalEncoder
ordinal_mappings = [
    ['Recreation', 'Fitness', 'Competition'],  # Reihenfolge für goal_of_training
    ['Morning', 'Afternoon', 'Evening']  # Reihenfolge für preferred_training_daytime
]

# Ordinal Encoding anwenden
ordinal_encoder = OrdinalEncoder(categories=ordinal_mappings)
df_step8[['goal_of_training', 'preferred_training_daytime']] = ordinal_encoder.fit_transform(
    df_step8[['goal_of_training', 'preferred_training_daytime']]
)

# 🔹 Ausgabe der Zuordnungen
print("\nZuordnungen für goal_of_training & preferred_training_daytime:")
for i, column in enumerate(['goal_of_training', 'preferred_training_daytime']):
    print(f"\n{column}:")
    for j, category in enumerate(ordinal_encoder.categories_[i]):
        print(f"{category} -> {j}")


In [ ]:
run_logistic_regression(df_step8, '8 Ordinal Encoding')
df_score_test

# Step9 Feature Selection 

In [ ]:
df_step9 = df_step0.copy()

Führt ANOVA F-Test durch, um die statistische Signifikanz der Features zu bewerten. 
Nutzt Random Forest Feature Importance für eine modellbasierte Bewertung.
Erstellt eine Korrelationsmatrix zur Identifizierung redundanter Features.
Kombiniert die Ergebnisse zu einer Empfehlung.
Behält nur die wichtigsten Features im finalen DataFrame.
Die Entscheidung, welche Features behalten werden sollen, basiert auf:
* Hoher Feature Importance
* Statistischer Signifikanz (niedriger p-Wert im ANOVA Test)
* Geringer Korrelation mit anderen Features

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
import numpy as np


# Features und Target definieren
X = df_step9[numeric_ohne_target]  # Numerische Features ohne Label
y = df_step9['user_of_latest_model']  # Zielvariable

# 1. ANOVA F-Test für numerische Features
f_selector = SelectKBest(f_classif, k='all')
f_selector.fit(X, y)

# Feature Scores aus ANOVA F-Test
anova_scores = pd.DataFrame({
    'Feature': X.columns,
    'F_Score': f_selector.scores_,
    'P_value': f_selector.pvalues_
})
anova_scores = anova_scores.sort_values('F_Score', ascending=False)

print("ANOVA F-Test Scores:")
print(anova_scores)

# 2. Random Forest Feature Importance
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)

# Feature Importance aus Random Forest
rf_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf.feature_importances_
})
rf_importance = rf_importance.sort_values('Importance', ascending=False)

print("\nRandom Forest Feature Importance:")
print(rf_importance)

# 3. Korrelationsmatrix für numerische Features
correlation_matrix = X.corr()

# Identifiziere stark korrelierte Features (z.B. > 0.8)
high_correlation = np.where(np.abs(correlation_matrix) > 0.8)
high_correlation = [(correlation_matrix.index[x], correlation_matrix.columns[y], correlation_matrix.iloc[x, y]) 
                   for x, y in zip(*high_correlation) if x != y and x < y]

print("\nStark korrelierte Features (>0.8):")
for feat1, feat2, corr in high_correlation:
    print(f"{feat1} - {feat2}: {corr:.2f}")

# Kombiniere die Ergebnisse für eine Empfehlung
# Wähle Features basierend auf Importance und geringer Korrelation
important_features = rf_importance[rf_importance['Importance'] > rf_importance['Importance'].mean()]['Feature'].tolist()

print("\nEmpfohlene Features basierend auf Importance und geringer Korrelation:")
print(important_features)

# Optional: Erstelle einen neuen DataFrame nur mit den wichtigsten Features
selected_features = important_features + ['user_of_latest_model']  # Füge Label hinzu
df_step9 = df_step9[selected_features]

# Visualisiere die Feature Importance
plt.figure(figsize=(10, 6))
plt.bar(rf_importance['Feature'], rf_importance['Importance'])
plt.xticks(rotation=45, ha='right')
plt.title('Feature Importance aus Random Forest')
plt.tight_layout()
plt.show()

Hier nun die Feature Selection der kategorischen Variablen:
Nutzt die bereits vorhandenen kategorischen Spalten (categorical_columns)
Führt das Ordinal Encoding durch
Berechnet die Feature Importance für kategorische Features
Visualisiert die Ergebnisse
Wählt wichtige kategorische Features aus
Kombiniert sie mit den bereits ausgewählten numerischen Features

In [ ]:
# Zurück zum ursprünglichen DataFrame für die kategorische Feature Selection, da im vorherigen Schritt die numerischen ausgesondert wurden
df_temp = df_step0.copy()

# Encoding für kategorische Features
encoder = OrdinalEncoder()
X_cat = pd.DataFrame(encoder.fit_transform(df_temp[categorical_columns]), 
                    columns=categorical_columns)
y_cat = df_temp['user_of_latest_model']

# Random Forest für kategorische Features
rf_cat = RandomForestClassifier(n_estimators=100, random_state=42)
rf_cat.fit(X_cat, y_cat)

# Feature Importance für kategorische Features
rf_importance_cat = pd.DataFrame({
    'Feature': X_cat.columns,
    'Importance': rf_cat.feature_importances_
})
rf_importance_cat = rf_importance_cat.sort_values('Importance', ascending=False)

print("\nRandom Forest Feature Importance (kategorische Features):")
print(rf_importance_cat)

# Wichtige kategorische Features auswählen
important_cat_features = rf_importance_cat[rf_importance_cat['Importance'] > 
                                         rf_importance_cat['Importance'].mean()]['Feature'].tolist()

print("\nWichtige kategorische Features:")
print(important_cat_features)

# Visualisierung der kategorischen Feature Importance
plt.figure(figsize=(10, 6))
plt.bar(rf_importance_cat['Feature'], rf_importance_cat['Importance'])
plt.xticks(rotation=45, ha='right')
plt.title('Feature Importance - Kategorische Features')
plt.tight_layout()
plt.show()

# Finalen DataFrame aktualisieren mit numerischen und kategorischen Features
all_selected_features = selected_features + important_cat_features
df_step9 = df_temp[all_selected_features]

print("\nFinale ausgewählte Features (numerisch + kategorisch):")
print(df_step9.columns.tolist())

In [ ]:
run_logistic_regression(df_step9, '9 Feature Selection')
df_score_test

# Step 10 Feature Extraction

Standardisiert die numerischen Daten
Führt PCA durch
Analysiert die erklärte Varianz
Visualisiert die kumulierte erklärte Varianz
Wählt die optimale Anzahl von Komponenten
Erstellt einen neuen DataFrame mit:

PCA-Komponenten für numerische Features
Originalen kategorischen Features
Target-Variable

Schwellenwert für die erklärte Varianz (hier 90%) nach Bedarf anpassen.
Die Feature Loadings am Ende zeigen, welche originalen Features am wichtigsten für jede Hauptkomponente sind.

In [ ]:
df_step10 = df_step0.copy()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 1️⃣ Numerische Werte aus df_step10 extrahieren
X = df_step10.select_dtypes(include=['int64', 'float64']).drop(columns=['user_of_latest_model'], errors='ignore')
y = df_step10['user_of_latest_model']

# 2️⃣ Standardisierung der numerischen Daten (wichtig für PCA)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3️⃣ PCA anwenden (hier mit 2 Hauptkomponenten, kann angepasst werden)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# 4️⃣ PCA-Ergebnis in einen neuen DataFrame umwandeln
df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2'], index=df_step10.index)

# 5️⃣ Zielvariable wieder hinzufügen
df_pca['user_of_latest_model'] = y.values  

# 6️⃣ Ergebnis anzeigen
print("Erklärte Varianz:", pca.explained_variance_ratio_)
df_pca



In [ ]:
run_logistic_regression(df_pca, '10 Feature Extraction')
df_score_test

# Auswertung der Steps

In [ ]:
# Matplotlib für bessere Lesbarkeit konfigurieren
plt.figure(figsize=(12, 6))

# Zwei Linien plotten
plt.plot(df_score_test.index, df_score_test['Accuracy'], marker='o', label='Accuracy', color='blue')
plt.plot(df_score_test.index, df_score_test['F1-Score'], marker='o', label='F1-Score', color='red')

# Beschriftungen und Titel
plt.xlabel('Preprocessing Schritte')
plt.ylabel('Score')
plt.title('Entwicklung von Accuracy und F1-Score über die Preprocessing Schritte')

# Legende hinzufügen
plt.legend()

# Grid für bessere Lesbarkeit
plt.grid(True, linestyle='--', alpha=0.7)

# X-Achsen-Labels rotieren für bessere Lesbarkeit
plt.xticks(df_score_test.index, ['Step '+str(i) for i in df_score_test.index], rotation=45)

# Layout optimieren
plt.tight_layout()

# Plot anzeigen
plt.show()